In [1]:
import os

ESCA_dataset = {
    'esca': {
        'folder': os.path.join(*['/', 'home', 'paperspace', 'GitHub', 'Robustness_of_Esca_detection', 'Python_files', 'ESCA_dataset', 'esca']),
        'esca_foliage_over_healthy_bg': os.path.join(*['/', 'home', 'paperspace', 'GitHub', 'Robustness_of_Esca_detection', 'Python_files', 'ESCA_dataset', 'esca', 'esca_foliage_over_healthy_bg']),
        'masks': os.path.join(*['/', 'home', 'paperspace', 'GitHub', 'Robustness_of_Esca_detection', 'Python_files', 'ESCA_dataset', 'esca', 'masks']),
        'pictures': os.path.join(*['/', 'home', 'paperspace', 'GitHub', 'Robustness_of_Esca_detection', 'Python_files', 'ESCA_dataset', 'esca', 'pictures']),
        'SAM_masks': os.path.join(*['/', 'home', 'paperspace', 'GitHub', 'Robustness_of_Esca_detection', 'Python_files', 'ESCA_dataset', 'esca', 'SAM_masks'])
    },
    'healthy': {
        'folder': os.path.join(*['/', 'home', 'paperspace', 'GitHub', 'Robustness_of_Esca_detection', 'Python_files', 'ESCA_dataset', 'healthy']),
        'healthy_foliage_over_esca_bg': os.path.join(*['/', 'home', 'paperspace', 'GitHub', 'Robustness_of_Esca_detection', 'Python_files', 'ESCA_dataset', 'esca', 'healthy_foliage_over_esca_bg']),
        'masks': os.path.join(*['/', 'home', 'paperspace', 'GitHub', 'Robustness_of_Esca_detection', 'Python_files', 'ESCA_dataset', 'healthy', 'masks']),
        'pictures': os.path.join(*['/', 'home', 'paperspace', 'GitHub', 'Robustness_of_Esca_detection', 'Python_files', 'ESCA_dataset', 'healthy', 'pictures']),
        'SAM_masks': os.path.join(*['/', 'home', 'paperspace', 'GitHub', 'Robustness_of_Esca_detection', 'Python_files', 'ESCA_dataset', 'healthy', 'SAM_masks'])
    }
}

In [2]:
import os

prms = {
    'checkpoint': 'sam_vit_h_4b8939.pth',
    'model-type': 'vit_h',
    'input': os.path.join(*['batch_segmentation_experiment', 'images', 'esca_001_cam1.jpg']),
    'output': os.path.join(*['batch_segmentation_experiment', 'generated_masks']),
    'device': 'cpu'
}
arg_string = f"--checkpoint {prms['checkpoint']} --model-type {prms['model-type']} --input {prms['input']} --output {prms['output']} --device {prms['device']}"
!python batch_segmentation.py {arg_string}

Loading model...
Traceback (most recent call last):
  File "/home/paperspace/GitHub/segment-anything/batch_segmentation.py", line 240, in <module>
    main(args)
  File "/home/paperspace/GitHub/segment-anything/batch_segmentation.py", line 199, in main
    sam = sam_model_registry[args.model_type](checkpoint=args.checkpoint)
  File "/home/paperspace/GitHub/segment-anything/segment_anything/build_sam.py", line 15, in build_sam_vit_h
    return _build_sam(
  File "/home/paperspace/GitHub/segment-anything/segment_anything/build_sam.py", line 104, in _build_sam
    with open(checkpoint, "rb") as f:
FileNotFoundError: [Errno 2] No such file or directory: 'sam_vit_h_4b8939.pth'


In [20]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

container_image = os.path.join(*['batch_segmentation_experiment', 'images', 'esca_000_cam1.jpg'])
img = plt.imread(container_image)
print(f"Dimensions of container image {container_image}: {img.shape}")
containing_folder = "/home/paperspace/GitHub/segment-anything/batch_segmentation_experiment/generated_masks/esca_000_cam1"
mask_file_list = os.listdir(containing_folder)
for filename in mask_file_list:
    full_name = os.path.join(*[folder, filename])
    msk = plt.imread(full_name)
    print(f"Dimensions of {full_name}: {msk.shape}")

def merge_binary_masks(containing_folder):
    result = None
    mask_file_list = os.listdir(containing_folder)
    # remember that normally there is a "metadata.csv" file inside containing_folder, that has been removed for this
    # experiment
    for count, filename in enumerate(mask_file_list):
        full_name = os.path.join(*[containing_folder, filename])
        msk = plt.imread(full_name)
        if count == 0:
            result = np.zeros(msk.shape)
        else:
            result = np.logical_or(result, msk)
    result_filename = os.path.join(*[containing_folder, 'final_merge.png'])
    Image.fromarray(result.astype('uint8')*255, mode='L').save(result_filename)

merge_binary_masks(containing_folder)


Dimensions of container image batch_segmentation_experiment/images/esca_000_cam1.jpg: (720, 1280, 3)
Dimensions of /home/paperspace/GitHub/segment-anything/batch_segmentation_experiment/generated_masks/esca_000_cam1/153.png: (720, 1280)
Dimensions of /home/paperspace/GitHub/segment-anything/batch_segmentation_experiment/generated_masks/esca_000_cam1/63.png: (720, 1280)
Dimensions of /home/paperspace/GitHub/segment-anything/batch_segmentation_experiment/generated_masks/esca_000_cam1/69.png: (720, 1280)
Dimensions of /home/paperspace/GitHub/segment-anything/batch_segmentation_experiment/generated_masks/esca_000_cam1/39.png: (720, 1280)
Dimensions of /home/paperspace/GitHub/segment-anything/batch_segmentation_experiment/generated_masks/esca_000_cam1/167.png: (720, 1280)
Dimensions of /home/paperspace/GitHub/segment-anything/batch_segmentation_experiment/generated_masks/esca_000_cam1/50.png: (720, 1280)
Dimensions of /home/paperspace/GitHub/segment-anything/batch_segmentation_experiment/ge

In [7]:
import json
import numpy as np
import matplotlib.pyplot as plt
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import os
# import sys
# sys.path.append("..")

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cpu"  # cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, fpga, ort, xla, lazy, vulkan, mps, meta, hpu, mtia, privateuseone

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)
img_files = os.listdir(ESCA_dataset['esca']['pictures'])
img_file = prms['input']
img = plt.imread(img_file)
masks = mask_generator.generate(img)

print(f"SAM detected {len(masks)} masks on image {img_file}")
# dict_keys(['segmentation', 'area', 'bbox', 'predicted_iou', 'point_coords', 'stability_score', 'crop_box'])
MAX = 3
stable_masks = 0
for count, mask in enumerate(masks):
    print(f" -------- mask {count} ---------")
    mask_relevant_data = {
        'area': mask['area'],
        'bbox': mask['bbox'],
        'predicted_iou': mask['predicted_iou'],
        'point_coords': mask['point_coords'],
        'stability_score': mask['stability_score'],
        'crop_box': mask['crop_box']
    }
    if count < MAX:
        json_object = json.dumps(mask_relevant_data, indent=2)
        print(json_object)
    if mask['stability_score'] > 0.8:
        stable_masks += 1

print(f"SAM detected {stable_masks} stable masks on image {img_file}")

SAM detected 173 masks on image batch_segmentation_experiment/images/esca_000_cam1.jpg
 -------- mask 0 ---------
{
  "area": 2738,
  "bbox": [
    0,
    0,
    102,
    36
  ],
  "predicted_iou": 1.0017647743225098,
  "point_coords": [
    [
      100.0,
      11.25
    ]
  ],
  "stability_score": 0.984425961971283,
  "crop_box": [
    0,
    0,
    1280,
    720
  ]
}
 -------- mask 1 ---------
{
  "area": 25221,
  "bbox": [
    967,
    591,
    290,
    127
  ],
  "predicted_iou": 0.9952616691589355,
  "point_coords": [
    [
      1220.0,
      663.75
    ]
  ],
  "stability_score": 0.9756488800048828,
  "crop_box": [
    0,
    0,
    1280,
    720
  ]
}
 -------- mask 2 ---------
{
  "area": 22142,
  "bbox": [
    779,
    101,
    219,
    142
  ],
  "predicted_iou": 0.990822970867157,
  "point_coords": [
    [
      860.0,
      101.25
    ]
  ],
  "stability_score": 0.9734296202659607,
  "crop_box": [
    0,
    0,
    1280,
    720
  ]
}
 -------- mask 3 ---------
 --------